In [21]:
# To visualize the data, we will use the pymatviz package
# ! pip install pymatviz

In [17]:
import numpy as np
import pandas as pd
from ast import literal_eval

from pymatgen.core import Structure
from pymatviz import StructureWidget
from pymatviz import structure_3d

from symmetry import find_spg
from pyxtal.symmetry import Group

In [85]:
#data = pd.read_csv('/home/user_wanglei/private/datafile/crystalgpt/csp/alex20s/csp-8c3cb/adam_bs_8000_lr_0.001_decay_0_clip_1_A_119_W_28_N_21_a_1_w_1_l_1_Nf_5_Kx_16_Kl_4_h0_256_l_16_H_16_k_64_m_64_e_32_drop_0.1_0.1//relaxed_structures_CsPbI3_200_ehull.csv')

#data = pd.read_csv('/home/user_wanglei/private/datafile/crystalgpt/csp/alex20s/csp-07d3f/adam_bs_8000_lr_0.0001_decay_0_clip_1_A_119_W_28_N_21_a_1_w_1_l_1_Nf_5_Kx_16_Kl_4_h0_256_l_16_H_16_k_64_m_64_e_32_drop_0.1_0.1//relaxed_structures_Cd3As2_4100_ehull.csv')

data = pd.read_csv('/home/user_wanglei/private/datafile/crystalgpt/csp/alex20s/csp-a20de/adam_bs_8000_lr_0.0001_decay_0_clip_1_A_119_W_28_N_21_a_1_w_1_l_1_Nf_5_Kx_16_Kl_4_h0_256_l_16_H_16_k_64_m_64_e_32_drop_0.1_0.1//relaxed_structures_Zr4O_8000_ehull.csv')

In [86]:
data = data.sort_values(by='relaxed_ehull', ascending=True).reset_index(drop=True)


In [87]:
cif_strings = data['relaxed_cif']   # string of pymatgen structure dict

In [88]:
structures =[Structure.from_dict(literal_eval(cif)) for cif in cif_strings]

In [ ]:
for i, struct in enumerate(structures):
    
    sg_num = find_spg(struct)
    g = Group(sg_num)
    # Get ehull values if available
    try:
        unrelaxed_ehull = data.loc[i, 'unrelaxed_ehull'] if 'unrelaxed_ehull' in data.columns else None
    except Exception:
        unrelaxed_ehull = None
    try:
        relaxed_ehull = data.loc[i, 'relaxed_ehull'] if 'relaxed_ehull' in data.columns else None
    except Exception:
        relaxed_ehull = None

    reduced_formula = struct.composition.reduced_formula
    num_atoms = len(struct)
    
    if reduced_formula == 'Zr4O':
        print(f"{i}: {reduced_formula} {num_atoms}, {sg_num} {g.symbol}, {unrelaxed_ehull:.6f}, {relaxed_ehull:.6f}")




In [29]:
structures[0]

Structure Summary
Lattice
    abc : 11.19291646729677 11.19304270772026 11.192662300357066
 angles : 109.47249776161688 109.47130765267586 109.47101956275652
 volume : 1079.4355061488884
      A : np.float64(6.462231150674858) np.float64(6.46226308062005) np.float64(6.462205774809851)
      B : np.float64(6.462340477428475) np.float64(-6.462318908358336) np.float64(-6.462259274999224)
      C : np.float64(-6.462286975316813) np.float64(6.462042965746097) np.float64(-6.461929829960737)
    pbc : True True True
PeriodicSite: As (-0.003151, -0.004903, -0.005137) [-0.0006231, 0.0001537, 1.809e-05]
PeriodicSite: As (6.463, 0.0009532, -6.464) [0.5001, 1.0, 0.5002]
PeriodicSite: As (6.461, 0.001988, -0.003536) [0.5, 0.5002, 0.0004275]
PeriodicSite: As (0.0008709, -0.002224, -6.465) [-9.606e-05, 0.5003, 0.5001]
PeriodicSite: As (3.44, 6.464, -3.229) [0.7663, 0.516, 0.75]
PeriodicSite: As (3.021, 6.46, 3.231) [0.7336, -0.01631, 0.2498]
PeriodicSite: As (3.233, 9.903, 0.002172) [1.016, 0.25, 0.7

In [44]:

structure_3d(structures[14], n_cols=1)

In [63]:
from pyxtal import pyxtal
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer

tol = 0.01
crystal = structures[250]
spga = SpacegroupAnalyzer(crystal, symprec=tol)
crystal = spga.get_refined_structure()
c = pyxtal()
c.from_seed(crystal, tol=tol)


In [64]:
c


------Crystal from Seed------
Dimension: 3
Composition: Zr24O6
Group: R -3:H (148)
  6.4810,   6.4810,  15.7066,  90.0000,  90.0000, 120.0000, trigonal
Wyckoff sites:
	Zr @ [ 0.0000  0.0000  0.2504], WP [6c] Site [3.]
	Zr @ [ 0.1712  0.3336  0.0832], WP [18f] Site [1]
	 O @ [ 0.0000  0.0000  0.0000], WP [3a] Site [-3.]
	 O @ [ 0.0000  0.0000  0.5000], WP [3b] Site [-3.]

In [58]:
structures[14]

Structure Summary
Lattice
    abc : 5.630786332522901 5.631393626720059 6.184492061317967
 angles : 62.91490698952506 62.91727647392077 60.001971488160834
 volume : 144.47027271457284
      A : np.float64(-2.815461088701051) np.float64(-4.8763647902056) np.float64(-0.00011567231552084923)
      B : np.float64(-5.631393617146569) np.float64(0.00027221187472897205) np.float64(-0.00018364335014383617)
      C : np.float64(-2.8157883751621684) np.float64(-1.6253896658384945) np.float64(-5.260930176023246)
    pbc : True True True
PeriodicSite: O (-5.63, -6.503, -5.261) [1.0, -0.0002436, 1.0]
PeriodicSite: Zr (-6.597, -1.627, -3.913) [0.08567, 0.7567, 0.7438]
PeriodicSite: Zr (-6.555, -4.899, -3.913) [0.7567, 0.4138, 0.7438]
PeriodicSite: Zr (-3.743, -3.227, -3.913) [0.4138, 0.08583, 0.7438]
PeriodicSite: Zr (-4.666, -4.874, -1.348) [0.9141, 0.2434, 0.2562]
PeriodicSite: Zr (-4.707, -1.603, -1.348) [0.2434, 0.5861, 0.2562]
PeriodicSite: Zr (-7.521, -3.275, -1.348) [0.5862, 0.9144, 0.2562]

In [ ]:
cif_string="""# generated using pymatgen
data_TiO2
_symmetry_space_group_name_H-M   'P 1'
_cell_length_a   3.78253968
_cell_length_b   3.78253968
_cell_length_c   9.61502157
_cell_angle_alpha   90.00000000
_cell_angle_beta   90.00000000
_cell_angle_gamma   90.00000000
_symmetry_Int_Tables_number   1
_chemical_formula_structural   TiO2
_chemical_formula_sum   'Ti4 O8'
_cell_volume   137.56794451
_cell_formula_units_Z   4
loop_
 _symmetry_equiv_pos_site_id
 _symmetry_equiv_pos_as_xyz
  1  'x, y, z'
loop_
 _atom_type_symbol
 _atom_type_oxidation_number
  Ti4+  4.0
  O2-  -2.0
loop_
 _atom_site_type_symbol
 _atom_site_label
 _atom_site_symmetry_multiplicity
 _atom_site_fract_x
 _atom_site_fract_y
 _atom_site_fract_z
 _atom_site_occupancy
  Ti4+  Ti0  1  0.50000000  0.50000000  0.50000000  1
  Ti4+  Ti1  1  0.50000000  0.00000000  0.75000000  1
  Ti4+  Ti2  1  0.00000000  0.00000000  0.00000000  1
  Ti4+  Ti3  1  0.00000000  0.50000000  0.25000000  1
  O2-  O4  1  0.00000000  0.50000000  0.45715213  1
  O2-  O5  1  0.50000000  0.50000000  0.70715213  1
  O2-  O6  1  0.50000000  0.00000000  0.54284787  1
  O2-  O7  1  0.00000000  0.00000000  0.79284788  1
  O2-  O8  1  0.50000000  0.00000000  0.95715213  1
  O2-  O9  1  0.00000000  0.00000000  0.20715212  1
  O2-  O10  1  0.00000000  0.50000000  0.04284788  1
  O2-  O11  1  0.50000000  0.50000000  0.29284788  1"""

In [ ]:
from pymatgen.io.cif import CifParser
parser = CifParser.from_str(cif_string)
crystal = parser.get_structures()[0]

In [ ]:
spga = SpacegroupAnalyzer(crystal, symprec=0.01)
c = pyxtal()
c.from_seed(crystal, tol=0.01)

In [ ]:
c

In [27]:
StructureWidget(structures[0])

StructureWidget(structure={'@module': 'pymatgen.core.structure', '@class': 'Structure', 'charge': 0.0, 'lattic…